In [ ]:
!pip install torch==2.1.1 torchvision==0.16.1 torchaudio==2.1.1 --index-url https://download.pytorch.org/whl/cu121
!pip install transformers accelerate huggingface_hub pandas

In [ ]:
!curl -L -o data/im2gps3k/im2gps3k_places365.csv \
     https://raw.githubusercontent.com/TIBHannover/GeoEstimation/original_tf/meta/im2gps3k_places365.csv

In [ ]:
from huggingface_hub import hf_hub_download, login

login(token="API_KEY")


# Specify the repository
repo_id = "Jia-py/MP16-Pro"

# List of files to download
files = [
    "metadata/MP16_Pro_filtered.csv",
    "metadata/MP16_Pro_places365.csv",
    "metadata/mp16_urls.csv",
    "mp-16-images00"
]

# Download each file
for file in files:
    try:
        file_path = hf_hub_download(repo_id=repo_id, filename=file, repo_type="dataset", local_dir="/data/mp16/")
        print(f"Downloaded {file} to {file_path}")
    except Exception as e:
        print(f"Error downloading {file}: {e}")



In [ ]:
!tar -xf /data/mp16/mp-16-images00

!cd /data/mp16/images && \
 ls > /data/mp16/all_files.txt && \
 total=$(wc -l < /data/mp16/all_files.txt) && \
 part=$((total / 10)) && \
 head -n "$part" /data/mp16/all_files.txt > /data/mp16/part_files.txt && \
 tar -cf /data/mp16/mp-16-images.tar -T /data/mp16/part_files.txt

In [ ]:
!python tune.py